# Sequence Models and LSTM Networks
Ref: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3,3)    # input dim is 3 and output dim is 3

In [3]:
inputs = [torch.randn(1,3) for _ in range(5)]    # make a sequence of length 5

In [4]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

In [10]:
print(inputs[1].shape)
print(inputs[1].view(1,1,-1).shape)

torch.Size([1, 3])
torch.Size([1, 1, 3])


In [17]:
# LSTM expects 3D tensor, hence view reshapes the input from (1,3) to (1,1,3)
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out.data, hidden)

tensor([[[-0.1787,  0.0873, -0.0086]]]) (tensor([[[-0.1787,  0.0873, -0.0086]]], grad_fn=<StackBackward>), tensor([[[-0.9052,  0.2019, -0.0194]]], grad_fn=<StackBackward>))
tensor([[[-0.5195,  0.0510,  0.0389]]]) (tensor([[[-0.5195,  0.0510,  0.0389]]], grad_fn=<StackBackward>), tensor([[[-1.0689,  0.1877,  0.0583]]], grad_fn=<StackBackward>))
tensor([[[-0.3853,  0.0557,  0.0270]]]) (tensor([[[-0.3853,  0.0557,  0.0270]]], grad_fn=<StackBackward>), tensor([[[-1.0424,  0.1691,  0.0483]]], grad_fn=<StackBackward>))
tensor([[[-0.1826,  0.0824,  0.0208]]]) (tensor([[[-0.1826,  0.0824,  0.0208]]], grad_fn=<StackBackward>), tensor([[[-1.0451,  0.2134,  0.0367]]], grad_fn=<StackBackward>))
tensor([[[-0.3585,  0.0879,  0.1488]]]) (tensor([[[-0.3585,  0.0879,  0.1488]]], grad_fn=<StackBackward>), tensor([[[-1.1368,  0.4596,  0.1772]]], grad_fn=<StackBackward>))


In [11]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


# An LSTM for Part-of-Speech Tagging

## Prepare data

In [25]:
def prepare_sequence(seq, to_ix):
    """Creates key-value pair of tokens to indices"""
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [26]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In [28]:
word_to_ix = {}
for sent, tags in training_data:
    print(sent, tags)
    for word in sent:
        print(word)
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

['The', 'dog', 'ate', 'the', 'apple'] ['DET', 'NN', 'V', 'DET', 'NN']
The
dog
ate
the
apple
['Everybody', 'read', 'that', 'book'] ['NN', 'V', 'DET', 'NN']
Everybody
read
that
book
{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [66]:
ix_to_tag = {0:"DET", 1:"NN", 2:"V"}
tag_to_ix = {"DET":0, "NN":1, "V":2}

### Create LSTM model

In [68]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Initialise the model

In [100]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 3
HIDDEN_DIM = 3

In [101]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

### Train the model

In [102]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.1437, -1.3792, -0.8449],
        [-1.1230, -1.3991, -0.8489],
        [-1.1139, -1.3738, -0.8709],
        [-1.1274, -1.4550, -0.8148],
        [-1.1259, -1.4758, -0.8051]])


In [103]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [104]:
# helper function
def tag_score_to_tag_name(tag_score, ix_to_tag):
    """Converts tag score to tag names"""
    return ix_to_tag.get(torch.argmax(tag_score).item())

In [105]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    
    print(f'Tag Scores:\n{tag_scores}\n')
    for i, token in enumerate(training_data[0][0]):
        print(f'{token:<10} {tag_score_to_tag_name(tag_scores[i], ix_to_tag)}')

Tag Scores:
tensor([[-0.1587, -2.4392, -2.8220],
        [-2.5748, -0.1373, -2.9541],
        [-2.2967, -2.5985, -0.1923],
        [-0.1951, -2.7746, -2.1642],
        [-2.8243, -0.0709, -4.7033]])

The        DET
dog        NN
ate        V
the        DET
apple      NN
